In [2]:
import json
from pathlib import Path
# Transforms the metadata of the klevr format to one needed for nerfstudio

In [ ]:
PATH = Path("/data/vision/polina/projects/wmh/dhollidt/klevr_nesf/klevr_nesf/0/metadata.json")

with open(PATH) as f:
    data = json.load(f)

In [3]:
import numpy as np
from scipy.spatial.transform import Rotation

def blender_quat2rot(quaternion):
  """Convert quaternion to rotation matrix.
  Equivalent to, but support batched case:
  ```python
  rot3x3 = mathutils.Quaternion(quaternion).to_matrix()
  ```
  Args:
    quaternion:
  Returns:
    rotation matrix
  """
  quaternion = np.asarray(quaternion)

  # Note: Blender first cast to double values for numerical precision while
  # we're using float32.
  q = np.sqrt(2) * quaternion

  q0 = q[..., 0]
  q1 = q[..., 1]
  q2 = q[..., 2]
  q3 = q[..., 3]

  qda = q0 * q1
  qdb = q0 * q2
  qdc = q0 * q3
  qaa = q1 * q1
  qab = q1 * q2
  qac = q1 * q3
  qbb = q2 * q2
  qbc = q2 * q3
  qcc = q3 * q3

  # Note: idx are inverted as blender and numpy convensions do not
  # match (x, y) -> (y, x)
  rotation = np.empty((*quaternion.shape[:-1], 3, 3), dtype=np.float32)
  rotation[..., 0, 0] = 1.0 - qbb - qcc
  rotation[..., 1, 0] = qdc + qab
  rotation[..., 2, 0] = -qdb + qac

  rotation[..., 0, 1] = -qdc + qab
  rotation[..., 1, 1] = 1.0 - qaa - qcc
  rotation[..., 2, 1] = qda + qbc

  rotation[..., 0, 2] = qdb + qac
  rotation[..., 1, 2] = -qda + qbc
  rotation[..., 2, 2] = 1.0 - qaa - qbb
  return rotation

def position_quaternion_to_transformation_matrix(position, quaternion):
    # Create a rotation object from the quaternion
    rotation_matrix = blender_quat2rot(quaternion)
    
    # Get the rotation matrix from the rotation object
    # rotation_matrix = rotation.as_matrix()
    
    # Combine the rotation matrix and the position vector into a transformation matrix
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix
    transformation_matrix[:3, 3] = position
    
    return transformation_matrix


def meta_data_to_nerfstudio_transform(metadata):
    aabb_scale = metadata["scene_boundaries"]["max"][0] - metadata["scene_boundaries"]["min"][0] 
    camera_angle_x = metadata["camera"]["field_of_view"]
    cx = metadata["metadata"]["width"] / 2
    cy = metadata["metadata"]["height"] / 2
    sensor_width = metadata["camera"]["sensor_width"]
    focal_length = metadata["camera"]["focal_length"]
    h = metadata["camera"]["height"]
    w = metadata["camera"]["width"]
    fl_x = focal_length * h / sensor_width
    fl_y = focal_length * w / sensor_width
    
    transformation_matrices = [position_quaternion_to_transformation_matrix(position, quaternion)
                               for position, quaternion in zip(metadata["camera"]["positions"], metadata["camera"]["quaternions"])]
    frame_list = [
        {
            "file_path": f"rgba_{i:05d}.png",
            "depth_file_path": f"depth_{i:05d}.tiff",
            "transform_matrix": transformation_matrix.tolist(),
        }
        for i, transformation_matrix in enumerate(transformation_matrices)
    ]
    return {
        "aabb_scale": aabb_scale,
        "camera_angle_x": camera_angle_x,
        "cx": cx,
        "cy": cy,
        "fl_x": fl_x,
        "fl_y": fl_y,
        "w": w,
        "h": h,
        "frames": frame_list,
    }
    

In [49]:
transforms = meta_data_to_nerfstudio_transform(data)

# save the dict as json to a file
OUTPATH = Path("/data/vision/polina/projects/wmh/dhollidt/klevr_nesf/klevr_nesf/0/transforms.json")
with open(OUTPATH, "w") as outfile:
    json.dump(transforms, outfile, indent=4)

In [4]:
IN_FOLDER_PATH = Path("/data/vision/polina/projects/wmh/dhollidt/datasets/toybox-5")

# iterate over all folders in the input folder
for folder in IN_FOLDER_PATH.iterdir():
    # check if the folder contains a metadata.json file
    if (folder / "metadata.json").is_file():
        # load the metadata.json file
        with open(folder / "metadata.json") as f:
            data = json.load(f)
        # convert the metadata to the nerfstudio format
        transforms = meta_data_to_nerfstudio_transform(data)
        # save the dict as json to a file
        with open(folder / "transforms.json", "w") as outfile:
            json.dump(transforms, outfile, indent=4)

In [48]:
for transform in transforms["frames"]:
    print(transform["transform_matrix"])

[[-0.6563112735748291, -0.2909194827079773, 0.6961474418640137, 7.481130123138428], [0.7544901371002197, -0.25306326150894165, 0.6055604815483093, 6.5076398849487305], [2.638547513811318e-08, 0.9226726293563843, 0.3855842351913452, 4.143670082092285], [0.0, 0.0, 0.0, 1.0]]
[[-0.6563113331794739, -0.35794076323509216, 0.6641790866851807, 7.481130123138428], [0.7544901967048645, -0.3113633394241333, 0.5777520537376404, 6.5076398849487305], [1.7304603971979304e-08, 0.8803017735481262, 0.4744141101837158, 5.343669891357422], [0.0, 0.0, 0.0, 1.0]]
[[-0.6563113331794739, -0.41558095812797546, 0.6297205090522766, 7.481130123138428], [0.7544901967048645, -0.36150309443473816, 0.5477774143218994, 6.5076398849487305], [2.6025885446756547e-08, 0.8346304893493652, 0.5508102774620056, 6.543670177459717], [0.0, 0.0, 0.0, 1.0]]
[[-0.6563112735748291, -0.4643932580947876, 0.5946379899978638, 7.481130123138428], [0.7544901967048645, -0.40396350622177124, 0.5172600746154785, 6.5076398849487305], [-1.655